In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

from sklearn import datasets
from sklearn.decomposition import PCA
from IPython.display import Image

Based on the tutorial presented here: http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html

## Iris Plants Database

This is perhaps the best known database to be found in the pattern recognition literature.  
The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant.  
One class is linearly separable from the other 2; the latter are NOT linearly separable from each other.  

We have measurements from 150 Iris flowers from 3 different species.

Attribute Information:
   1. sepal length in cm
   2. sepal width in cm
   3. petal length in cm
   4. petal width in cm
   5. class:  
      -- Iris Setosa  
      -- Iris Versicolour  
      -- Iris Virginica  

Sources:  
     (a) Creator: R.A. Fisher  
     (b) Donor: Michael Marshall (MARSHALL%PLU@io.arc.nasa.gov)  
     (c) Date: July, 1988  
     


In [ ]:
Image("images/iris_with_labels.jpg")

In [ ]:
Image("images/iris_types.jpg")

### Lets import the dataset and play around

In [ ]:
columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
df = pd.read_csv('ds/iris.data', header=None, names=columns)
df.dropna(how="all", inplace=True)

In [ ]:
df.head()

In [ ]:
df.groupby('class').mean()

### Next, we seperate the dataframe into class labels and an attribute matrix

In [ ]:
mat = df.iloc[:, :4]
classes = df.iloc[:, 4].values
mat.head()

### How does each feature vary across species?

In [ ]:
label_dict = {1: 'Iris-Setosa',
              2: 'Iris-Versicolor',
              3: 'Iris-Virgnica'}

feature_dict = {0: 'sepal_length',
                1: 'sepal_width',
                2: 'petal_length',
                3: 'petal_width'}

with plt.style.context('seaborn-darkgrid'):
    plt.figure(figsize=(16, 12))
    
    for cnt in range(4):
        plt.subplot(2, 2, cnt+1)
        for lab in ('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'):
            plt.hist(df[df['class'] == lab][feature_dict[cnt]],
                     label=lab,
                     bins=10,
                     alpha=0.9,)
        plt.xlabel(feature_dict[cnt])
    
    plt.legend(loc='upper right', fancybox=True, fontsize=8)

    plt.tight_layout()
    plt.show()

# A Summary of the PCA Approach
* Standardize the data.
* Obtain the Eigenvectors and Eigenvalues from the covariance matrix or correlation matrix, or perform Singular Vector Decomposition.
* Sort eigenvalues in descending order and choose the $k$ eigenvectors that correspond to the $k$ largest eigenvalues where $k$ is the number of dimensions of the new feature subspace ($k \leq d$).
* Construct the projection matrix $W$ from the selected $k$ eigenvectors.
* Transform the original dataset $X$ via $W$ to obtain a $k$-dimensional feature subspace $Y$.

In [ ]:
from sklearn.preprocessing import StandardScaler

## Step 1 - Preprocessing/Scaling
 We'll scale the data to have a mean of 0 and variance of 1  
 We can use the StandardScaler class from sklearn for this

In [ ]:
scaler = StandardScaler()
mat_std = scaler.fit_transform(mat)
print(mat.values[0])
print(mat_std[0])

## Step 2 - Eigendecomposition

The eigenvectors and eigenvalues of a covariance (or correlation) matrix represent the “core” of a PCA: The eigenvectors (principal components) determine the directions of the new feature space, and the eigenvalues determine their magnitude. In other words, the eigenvalues explain the variance of the data along the new feature axes.

The classic approach to PCA is to perform the eigendecomposition on the covariance matrix $\Sigma$, which is a 
$d \times d$ matrix where each element represents the covariance between two features. The covariance between two features is calculated as follows:

$$\sigma_{jk} = \frac{1}{n-1} (X - \overline{x})^T (X - \overline{x})$$

where $\overline{x}$ is the mean vector, $\frac{1}{n} \sum_{i=i}^{n} x_i$

The mean vector is a $d$-dimensional vector where each value in this vector represents the sample mean of a feature column in the dataset.

### We'll use the covariance matrix approach, but you could also use the correlation matrix or perform Singular Vector Decomposition

In [ ]:
mean_vec = np.mean(mat_std, axis=0) # by column
cov_mat = (mat_std - mean_vec).T.dot((mat_std - mean_vec)) / (mat_std.shape[0] - 1)
print(cov_mat)

#### Is there a library for that?

In [ ]:
cov_mat = np.cov(mat_std.T)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

In [ ]:
print(eig_vals)

In [ ]:
print(eig_vecs.T)

## Step 3 - Selecting Principal Components

In order to decide which eigenvector(s) can dropped without losing too much information for the construction of lower-dimensional subspace, we need to inspect the corresponding eigenvalues: The eigenvectors with the lowest eigenvalues bear the least information about the distribution of the data; those are the ones can be dropped.
In order to do so, the common approach is to rank the eigenvalues from highest to lowest in order choose the top $k$
eigenvectors.

In [ ]:
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs.sort(key=lambda x: x[0], reverse=True)

In [ ]:
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0],": ",i[1])

### Explained Variance
After sorting the eigenpairs, the next question is “how many principal components are we going to choose for our new feature subspace?” A useful measure is the so-called “explained variance,” which can be calculated from the eigenvalues. The explained variance tells us how much information (variance) can be attributed to each of the principal components.

In [ ]:
total = sum(eig_vals)
var_exp = [(i / total)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
cum_var_exp

In [ ]:
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(6, 4))

    plt.bar(range(4), var_exp, alpha=0.5, align='center',
            label='individual explained variance')
    plt.step(range(4), cum_var_exp, where='mid',
             label='cumulative explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()

The plot above clearly shows that most of the variance (72.77% of the variance to be precise) can be explained by the first principal component alone. The second principal component still bears some information (23.03%) while the third and fourth principal components can safely be dropped without losing to much information. Together, the first two principal components contain 95.8% of the information.

## Step 4 - Projection Matrix
The construction of the projection matrix that will be used to transform the Iris data onto the new feature subspace. Although, the name “projection matrix” has a nice ring to it, it is basically just a matrix of our concatenated top k eigenvectors.


Here, we are reducing the 4-dimensional feature space to a 2-dimensional feature subspace, by choosing the “top 2” eigenvectors with the highest eigenvalues to construct our $d \times k$-dimensional eigenvector matrix $W$.

In [ ]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(4,1),
                      eig_pairs[1][1].reshape(4,1)))
print(matrix_w)

## Step 5 - Projecting onto the new feature space

In this last step we will use the $4\times2$-dimensional projection matrix $W$ to transform our samples onto the new subspace via the equation
$$Y=X\times W$$
, where $Y$ is a $150\times 2$ matrix of our transformed samples.

In [ ]:
Y = mat_std.dot(matrix_w)
print(Y)

Now, what we have after applying the linear PCA transformation is a lower dimensional subspace (from 4D to 2D in this case), where the samples are “most spread” along the new feature axes.

In [ ]:
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(12, 8))
    for lab, col in zip(('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'),
                        ('blue', 'red', 'green')):
        plt.scatter(Y[classes == lab, 0],
                    Y[classes == lab, 1],
                    label=lab,
                    c=col)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend(loc='lower center')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(mat_std)

# 0,1 denote PC1 and PC2; change values for other PCs
xvector = sklearn_pca.components_[0] 
yvector = sklearn_pca.components_[1]

xs = sklearn_pca.transform(mat_std)[:,0] 
ys = sklearn_pca.transform(mat_std)[:,1]




In [ ]:
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(12, 8))
    for lab, col in zip(('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'),
                        ('blue', 'red', 'green')):
        plt.scatter(Y_sklearn[classes==lab, 0],
                    Y_sklearn[classes==lab, 1],
                    label=lab,
                    c=col)
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend(loc='lower right')
    plt.tight_layout()
    
    
    for i in range(len(xvector)):
# arrows project features (ie columns from csv) as vectors onto PC axes
        plt.arrow(0, 0, xvector[i]*max(xs), yvector[i]*max(ys),
              color='r', width=0.0005, head_width=0.0025)
    
        plt.text(xvector[i]*max(xs)*1.2, yvector[i]*max(ys)*1.2,
             list(df.columns.values)[i], color='black')
    
    plt.show()

In [ ]:
print(xvector)
print(yvector)